# Chess Engine with PyTorch

## Imports

In [1]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import DataLoader # type: ignore
from chess import pgn # type: ignore
from tqdm import tqdm # type: ignore

# Data preprocessing

## Load data

In [2]:
def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

files = [file for file in os.listdir("../../data/pgn") if file.endswith(".pgn")]
print(len(files))
LIMIT_OF_FILES = min(len(files), 10)
games = []
i = 1
for file in tqdm(files):
    games.extend(load_pgn(f"../../data/pgn/{file}"))
    if i >= LIMIT_OF_FILES:
        break
    i += 1

79


 11%|█▏        | 9/79 [00:01<00:14,  4.81it/s]


In [3]:
print(f"GAMES PARSED: {len(games)}")

GAMES PARSED: 686


## Convert data into tensors

In [4]:
from auxiliary_func import create_input_for_nn, encode_moves, create_input_for_nn_2

In [5]:
X, y = create_input_for_nn_2(games)

print(f"NUMBER OF SAMPLES: {len(y)}")

NUMBER OF SAMPLES: 61619


In [6]:
X = X[0:2_500_000]
y = y[0:2_500_000]

In [7]:
y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

In [8]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Preliminary actions

In [29]:
from dataset import ChessDataset
import model as custom_model
import importlib
importlib.reload(custom_model)

<module 'model' from 'c:\\Users\\salla\\OneDrive\\Documents\\Cours\\A3\\Advanced_Machine_Learning\\Kaggle_Competitions\\chess_engine_main\\engines\\torch\\model.py'>

In [32]:
# Create Dataset and DataLoader
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Model Initialization
model = custom_model.ChessModel4(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: cpu


# Training

In [33]:
num_epochs = 100
# resume training
# model.load_state_dict(torch.load("../../models/TORCH_50EPOCHS_3.pth", map_location=device))
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()

        outputs = model(inputs)  # Raw logits

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        running_loss += loss.item()
    end_time = time.time()
    epoch_time = end_time - start_time
    minutes: int = int(epoch_time // 60)
    seconds: int = int(epoch_time) - minutes * 60
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(dataloader):.4f}, Time: {minutes}m{seconds}s')

100%|██████████| 963/963 [01:33<00:00, 10.25it/s]


Epoch 1/100, Loss: 6.2387, Time: 1m33s


100%|██████████| 963/963 [01:16<00:00, 12.59it/s]


Epoch 2/100, Loss: 5.3948, Time: 1m16s


100%|██████████| 963/963 [01:26<00:00, 11.17it/s]


...

Epoch 98/100, Loss: 0.2211, Time: 0m37s


100%|██████████| 963/963 [00:37<00:00, 25.42it/s]


Epoch 99/100, Loss: 0.2204, Time: 0m37s


100%|██████████| 963/963 [00:38<00:00, 25.30it/s]

Epoch 100/100, Loss: 0.2194, Time: 0m38s


# Save the model and mapping

In [34]:
# Save the model
torch.save(model.state_dict(), "../../models/TORCH_50EPOCHS_3.pth")

In [35]:
import pickle

with open("../../models/heavy_move_to_int3", "wb") as file:
    pickle.dump(move_to_int, file)